In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [3]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")


In [4]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


Saving the Real time audio File 

In [5]:
import pyaudio
import wave

# Parameters for audio recording
FORMAT = pyaudio.paInt16  # Format for audio recording
CHANNELS = 1  # Number of audio channels (1 for mono, 2 for stereo)
RATE = 44100  # Sample rate (samples per second)
CHUNK = 1024  # Size of each audio chunk
RECORD_SECONDS = 5  # Duration of the recording (in seconds)
OUTPUT_FILENAME = "output.wav"  # Output audio file

audio = pyaudio.PyAudio()

# Create an audio stream
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

print("Recording...")

frames = []

# Record audio in chunks and store in frames
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Finished recording.")


# Stop and close the audio stream
stream.stop_stream()
stream.close()

# Terminate the PyAudio instance

audio.terminate()

# Save the recorded audio to a WAV file
with wave.open(OUTPUT_FILENAME, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))

print(f"Audio saved to {OUTPUT_FILENAME}")


Recording...
Finished recording.
Audio saved to output.wav


Converting the Audio File to Text format

In [6]:
#import library
import speech_recognition as sr

# Initialize recognizer class (for recognizing the speech)
r = sr.Recognizer()

# Reading Audio file as source
# listening the audio file and store in audio_text variable

with sr.AudioFile('output.wav') as source:
    
    audio_text = r.listen(source)
    
# recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
    try:
        
        # using google speech recognition
        text = r.recognize_google(audio_text)
        print('Converting audio transcripts into text ...')
        print(text)
     
    except:
         print('Sorry.. run again...')


Converting audio transcripts into text ...
hello my name is Kevin Modi I am a student at univers


Speech to Text

In [7]:
import speech_recognition as sr

# Create a recognizer object
r = sr.Recognizer()

# Set the language code
r.language_code = 'en-US'

# Get the audio source
with sr.Microphone() as source:
    audio = r.listen(source)

# Recognize the speech
try:
    text = r.recognize_google(audio)
    print(text)
except sr.UnknownValueError:
    print('Could not hear or recognize speech')
except sr.RequestError:
    print('Could not connect to Google Speech Recognition service')


hello my name is Kevin Modi I am a student at Universal College of Engineering


Coverting the Text(English) to Hindi

In [8]:
model_inputs = tokenizer(text, return_tensors="pt")
# translate from English to Hindi
generated_tokens = model.generate(**model_inputs,forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

translation


['हेलो मेरा नाम है केविन मोदी मैं यूनिवर्सल कॉलेज ऑफ इंजीनियरिंग में एक छात्र हूँ']